In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain_groq import ChatGroq

In [3]:
pip install python-env

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()

True

In [6]:
KEY=os.getenv("GROQ_API_KEY")

In [7]:
llm=ChatGroq(groq_api_key=KEY,model_name="Gemma-7b-it")

In [8]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000261155DAF70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000261155E8BB0>, model_name='Gemma-7b-it')

In [9]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [11]:
TEMPLATE="""
Text: {text}

You are an expert MCQ creator. Based on the above text, your task is to create a quiz consisting of exactly {number} multiple-choice questions for {subject} students in a {tone} tone. 

Guidelines:
1. Ensure all questions are unique and directly related to the provided text.
2. Verify that all questions and answers conform to the information in the text.
3. Use the response  format provided below as a guide.
4. The response should be in a json format to convert it in dataframe.
5. Follow the same{response_json} format.


###RESPONSE_FORMAT
   {response_json}

"""

In [12]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [13]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4092\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [14]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [15]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [16]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [17]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [18]:
file_path=r"C:\Users\ACER\mcqq\data.txt"

In [19]:
file_path

'C:\\Users\\ACER\\mcqq\\data.txt'

In [20]:
with open(file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()

In [21]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [22]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [23]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [24]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON,indent=4)
        }
        )

C:\Users\ACER\AppData\Local\Temp\ipykernel_4092\3593349545.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied huma

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers wh

In [25]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3436
Prompt Tokens:2514
Completion Tokens:922
Total Cost:0.0


In [26]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [65]:
quiz=response.get("quiz")

In [67]:
import json

quiz_json = json.dumps(quiz, indent=4)  # Convert the dictionary to a pretty JSON string
type(quiz_json)


str

In [63]:
import json

# The 'quiz' JSON string from the response
quiz_json = response['quiz']#response.get("quiz")

# Step 1: Parse the JSON string into a Python dictionary
try:
    quiz = json.loads(quiz_json.strip('```json').strip('```'))
    print(type(quiz))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    quiz = {}

# Step 2: Preprocess the dictionary into a list of records
quiz_records = []
if isinstance(quiz, dict):
    for question_id, question_data in quiz.items():
        flat_question = {
            "question_id": question_id,
            "mcq": question_data["mcq"],
            "correct": question_data["correct"]
        }
        # Add options with flattened keys
        flat_question.update({
            f"option_{key}": value for key, value in question_data["options"].items()
        })
        quiz_records.append(flat_question)
else:
    print("Parsed 'quiz' is not a dictionary.")

# Step 3: Print the processed records for verification
for record in quiz_records:
    print(record)


<class 'dict'>
{'question_id': '1', 'mcq': "When was the term 'machine learning' coined?", 'correct': 'b', 'option_a': '1950s', 'option_b': '1959', 'option_c': '1960s', 'option_d': '1970s'}
{'question_id': '2', 'mcq': 'Who is credited with introducing the earliest machine learning model?', 'correct': 'a', 'option_a': 'Arthur Samuel', 'option_b': 'Donald Hebb', 'option_c': 'Raytheon Company', 'option_d': 'Walter Pitts and Warren McCulloch'}
{'question_id': '3', 'mcq': 'What is the primary goal of modern-day machine learning?', 'correct': 'c', 'option_a': 'Classifying data based on models', 'option_b': 'Predicting future outcomes', 'option_c': 'Both A and B', 'option_d': 'None of the above'}
{'question_id': '4', 'mcq': 'Who provided a formal definition of machine learning algorithms?', 'correct': 'b', 'option_a': 'Arthur Samuel', 'option_b': 'Tom M. Mitchell', 'option_c': 'Donald Hebb', 'option_d': 'Walter Pitts and Warren McCulloch'}
{'question_id': '5', 'mcq': 'What is the ultimate goa

In [69]:
# Step 4: Save to CSV (optional)
import pandas as pd

# Assuming 'quiz_records' is a list of dictionaries from the previous code
df = pd.DataFrame(quiz_records)

# Step 2: Display the DataFrame to verify the results
df.head()


,question_id,mcq,correct,option_a,option_b,option_c,option_d
0,1,When was the term 'machine learning' coined?,b,1950s,1959,1960s,1970s
1,2,Who is credited with introducing the earliest ...,a,Arthur Samuel,Donald Hebb,Raytheon Company,Walter Pitts and Warren McCulloch
2,3,What is the primary goal of modern-day machine...,c,Classifying data based on models,Predicting future outcomes,Both A and B,None of the above
3,4,Who provided a formal definition of machine le...,b,Arthur Samuel,Tom M. Mitchell,Donald Hebb,Walter Pitts and Warren McCulloch
4,5,What is the ultimate goal of machine learning ...,b,For machines to exhibit human-like consciousness,For machines to perform tasks that require hum...,For machines to understand human language,For machines to replicate biological functions


In [70]:
# Step 3: Optionally, save the DataFrame to a CSV file if needed
df.to_csv('quiz_records.csv', index=False)
